In [13]:
import numpy as np
from keras.layers import Input, Dense
from keras.models import Sequential, load_model
import random

In [14]:
abecedario  =  [ "a"  ,"b" , "c" , "d" ,"e" ,  "f"  , "g"  , "h",   "i" ,"j","k","l","m","n","o","p","q","r","s","t","u" ,"v" ,"w" ,"x"  , "y"  , "z" ,"-", "/" ,"á" ,"é" ,"í", "ó","ú","ñ","1" ,"2","3","4","5","6","7","8","9","0"]
numeracion =   [97 , 98 , 99 , 100  ,101  ,102  ,103 , 104 , 105,  106 , 107 , 108  ,109 , 110 , 111  ,112 , 113  ,114 , 115 , 116  ,117 , 118  ,119 , 120  ,121 , 122 , 45,47,160,130,161,162,163,164,49,50,51,52,53,54,55,56,57,48]   # numeracioen de esta pagina https://elcodigoascii.com.ar/codigos-ascii-extendidos/letra-e-minuscula-acento-agudo-codigo-ascii-130.html

def ASCIIfuncLetter(letter):
    d = 0
    for i in range(len(abecedario)):
        if letter == abecedario[i]:
            d = numeracion[i]
            return d
    return d

def ASCIIfuncNum(num):
  d = " "
  for i in range(len(numeracion)):
      if num == numeracion[i]:
          d = abecedario[i]
          return d
  return d


def TxttoNum(word):
  Num = []
  for i in range(len(word)):
      Num.append(ASCIIfuncLetter(word.lower()[i]))
  return Num

def NumtoTxt(num):
  Txt = ""
  for i in range(len(num)):
      Txt = Txt + ASCIIfuncNum(num[i])
  return Txt


In [15]:
with open('archivo_filtrado_MED.txt', 'r') as archivo:
    lineas = archivo.readlines()
poke = []
med = []
entrenamiento = []
for linea in lineas:
    linea = linea.replace('\n', '')
    entrenamiento.append(linea)
    med.append(linea)

with open('data.txt', 'w') as data:
    data.writelines("%s" % linea for linea in lineas)

UNO =  len(entrenamiento)

with open('archivo_filtrado_POK.txt', 'r') as archivo:
    lineas = archivo.readlines()

for linea in lineas:
    linea = linea.replace('\n', '')
    entrenamiento.append(linea)
    poke.append(linea)

with open('data.txt', 'w') as data:
    for i in range(len(entrenamiento)):
       data.writelines(entrenamiento[i] + "\n" )

copia = entrenamiento.copy()

etiquetas = []

for i in range(UNO):
    etiquetas.append(1)

for i in range(len(entrenamiento)-UNO):
    etiquetas.append(0)

# Empaquetar los datos y etiquetas juntos
datos_con_etiquetas = list(zip(entrenamiento, etiquetas))

# Barajar los datos
random.shuffle(datos_con_etiquetas)

# Separar los datos y las etiquetas una vez barajados
entrenamiento, etiquetas = zip(*datos_con_etiquetas)

# Guardar los datos en el archivo data.txt
with open('data.txt', 'w') as data:
    for linea in entrenamiento:
        data.write(linea + "\n")


# Dividir los datos en 90% entrenamiento y 10% validación
tamano_validacion = int(0.1 * len(entrenamiento))

# Conjunto de validación (10%)
entrenamiento_validacion = entrenamiento[:tamano_validacion]
etiquetas_validacion = etiquetas[:tamano_validacion]

# Conjunto de entrenamiento (90%)
entrenamiento = entrenamiento[tamano_validacion:]
etiquetas_entrenamiento = etiquetas[tamano_validacion:]

# Guardar los datos de entrenamiento en un archivo
with open('data.txt', 'w') as data:
    for linea in entrenamiento:
        data.write(linea.lower() + "\n")

with open('data_etiqueta.txt', 'w') as data:
    for linea in etiquetas_entrenamiento:
        data.write(str(linea) + "\n")

# Guardar los datos de validación en un archivo
with open('data_validacion.txt', 'w') as data:
    for linea in entrenamiento_validacion:
        data.write(linea.lower() + "\n")

with open('data_validacion_etiqueta.txt', 'w') as data:
    for linea in etiquetas_validacion:
        data.write(str(linea) + "\n")



In [16]:
len_max = 0

for i in range(len(entrenamiento)):
    if len(copia[i]) > len_max:
        len_max = len(copia[i])

data = []
for i in range(len(entrenamiento)):
    Num = TxttoNum(entrenamiento[i])
    if len(entrenamiento[i]) < len_max:
      for j in range(len_max - len(entrenamiento[i])):
        Num.append(0)
    data.append(Num)

data_validacion = []
for i in range(len(entrenamiento_validacion)):
    Num = TxttoNum(entrenamiento_validacion[i])
    if len(entrenamiento_validacion[i]) < len_max:
      for j in range(len_max - len(entrenamiento_validacion[i])):
        Num.append(0)
    data_validacion.append(Num)

In [17]:
print(len_max)

38


In [18]:
n = 30 # cantidad de redes

step_layer = 5 # cantidad de nodos que se agregan por capa

In [19]:
 # Dataset de ejemplo
palabras = data  # Lista de palabras

etiquetas = etiquetas_entrenamiento  # 0: Pokémon, 1: Medicamento

palabras = np.array(palabras, dtype='float32')
etiquetas = np.array(etiquetas, dtype='float32')


# Convertir etiquetas en numpy array
y = np.array(etiquetas)

for i in range(n):

  # Definir el modelo de la red neuronal
  model = Sequential()
  model.add(Dense(10 + i*step_layer, input_dim=len_max, activation='relu'), )
  model.add(Dense(10 + i*step_layer, activation='relu'))
  model.add(Dense(10 + i*step_layer, activation='relu'))
  model.add(Dense(1,  activation='sigmoid'))  # Clasificación binaria



  # Compilar el modelo
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Entrenar el modelo
  model.fit(palabras, etiquetas, epochs=4000)

  model.save("red_POKMED"  + str(10 + i*step_layer) + ".keras")

Epoch 1/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4236 - loss: 7.6031
Epoch 2/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4372 - loss: 1.3166
Epoch 3/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5075 - loss: 0.7514
Epoch 4/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5678 - loss: 0.6830
Epoch 5/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6386 - loss: 0.6597
Epoch 6/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6656 - loss: 0.6450
Epoch 7/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6802 - loss: 0.6329
Epoch 8/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6845 - loss: 0.6347
Epoch 9/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6849 - loss: 0.6116
Epoch 10/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7000 - loss: 0.6080
Epoch 11/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6822 - loss: 0.6140
Epoch 12/4000
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

KeyboardInterrupt: 